In [19]:
import tensorflow as tf
from tensorflow.keras import datasets,layers,models
import matplotlib.pyplot as plt

In [20]:
#download the CIFAR 100 datasets 
(x_train,y_train),(x_test,y_test)=datasets.cifar100.load_data()

#normalize the data
x_train,x_test=x_train/255.0,x_test/255.0

In [21]:
from sklearn.model_selection import train_test_split
x_train,x_val,y_train,y_val=train_test_split(x_train,y_train,test_size=0.2)

In [22]:
x_train.shape


(40000, 32, 32, 3)

In [23]:
from keras.utils import  to_categorical
y_train=to_categorical(y_train)
y_test=to_categorical(y_test)
y_val=to_categorical(y_val)

In [24]:
## VGG16  with the sgd and no regularization
import keras,os
from keras.models import Sequential
from keras.layers import Dense,MaxPool2D,Conv2D,Flatten
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

model=Sequential()
model.add(Conv2D(input_shape=(32,32,3),filters=64,kernel_size=(3,3),activation="relu",padding="same"))
model.add(Conv2D(filters=64,kernel_size=(3,3),activation="relu",padding="same"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

model.add(Conv2D(filters=128,kernel_size=(3,3),activation="relu",padding="same"))
model.add(Conv2D(filters=128,kernel_size=(3,3),activation="relu",padding="same"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

model.add(Conv2D(filters=256,kernel_size=(3,3),activation="relu",padding="same"))
model.add(Conv2D(filters=256,kernel_size=(3,3),activation="relu",padding="same"))
model.add(Conv2D(filters=256,kernel_size=(3,3),activation="relu",padding="same"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

model.add(Conv2D(filters=512,kernel_size=(3,3),activation="relu",padding="same"))
model.add(Conv2D(filters=512,kernel_size=(3,3),activation="relu",padding="same"))
model.add(Conv2D(filters=512,kernel_size=(3,3),activation="relu",padding="same"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))


model.add(Conv2D(filters=512,kernel_size=(3,3),activation="relu",padding="same"))
model.add(Conv2D(filters=512,kernel_size=(3,3),activation="relu",padding="same"))
model.add(Conv2D(filters=512,kernel_size=(3,3),activation="relu",padding="same"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

model.add(Flatten())
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=100,activation="softmax"))



In [25]:
from keras.optimizers import  SGD

model.compile(optimizer=SGD(lr=0.01),loss='categorical_crossentropy',metrics=["accuracy"])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 32, 32, 64)        1792      
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 32, 32, 64)        36928     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 16, 16, 128)       73856     
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 16, 16, 128)       147584    
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 8, 8, 128)         0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 8, 8, 256)        

In [26]:
#early stopping
from keras.callbacks import EarlyStopping
early=EarlyStopping(monitor='val_loss',min_delta=0.0001,patience=30,verbose=1,mode='auto')

In [27]:
history=model.fit(x_train,y_train,batch_size=50,validation_data=(x_val,y_val),epochs=1000,callbacks=[early])

Epoch 1/15
800/800 [==============================] - 30s 37ms/step - loss: 4.6053 - accuracy: 0.0093 - val_loss: 4.6054 - val_accuracy: 0.0080
Epoch 2/15
800/800 [==============================] - 30s 38ms/step - loss: 4.6052 - accuracy: 0.0102 - val_loss: 4.6056 - val_accuracy: 0.0080
Epoch 3/15
800/800 [==============================] - 30s 37ms/step - loss: 4.6052 - accuracy: 0.0098 - val_loss: 4.6057 - val_accuracy: 0.0080
Epoch 4/15
800/800 [==============================] - 30s 37ms/step - loss: 4.6051 - accuracy: 0.0103 - val_loss: 4.6059 - val_accuracy: 0.0080
Epoch 5/15
800/800 [==============================] - 30s 37ms/step - loss: 4.6051 - accuracy: 0.0105 - val_loss: 4.6060 - val_accuracy: 0.0080
Epoch 6/15
800/800 [==============================] - 30s 37ms/step - loss: 4.6050 - accuracy: 0.0105 - val_loss: 4.6063 - val_accuracy: 0.0080
Epoch 8/15
800/800 [==============================] - 30s 37ms/step - loss: 4.6050 - accuracy: 0.0104 - val_loss: 4.6064 - val_accuracy:

In [28]:
# predict the 

score=model.predict(x_test).argmax(-1)


In [29]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score,accuracy_score,recall_score
matrix1=confusion_matrix(y_test.argmax(axis=1),score)
print(matrix1)
p_score=precision_score(y_test.argmax(axis=1),score,average='weighted')
acc_score=accuracy_score(y_test.argmax(axis=1),score)
r_score=recall_score(y_test.argmax(axis=1),score,average='weighted')

print('precision',p_score)
print('accuracy',acc_score)
print('recall',r_score)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
precision 0.0001526682409673071
accuracy 0.0101
recall 0.0101


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [114]:
#  #Vgg16 with adams


# (x_train,y_train),(x_test,y_test)=datasets.cifar100.load_data()

# #normailze the data
# x_train,x_test=x_train/255.0,x_test/255.0



# from sklearn.model_selection import train_test_split
# x_train,x_val,y_train,y_val=train_test_split(x_train,y_train,test_size=0.2)

# from keras.utils import  to_categorical
# y_train=to_categorical(y_train)
# y_test=to_categorical(y_test)
# y_val=to_categorical(y_val)


# import keras,os
# from keras.models import Sequential
# from keras.layers import Dense,MaxPool2D,Conv2D,Flatten,BatchNormalization
# from keras.preprocessing.image import ImageDataGenerator
# import numpy as np

# model=Sequential()
# model.add(Conv2D(input_shape=(32,32,3),filters=64,kernel_size=(3,3),activation="relu",padding="same"))
# model.add(Conv2D(filters=64,kernel_size=(3,3),activation="relu",padding="same"))
# model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

# model.add(Conv2D(filters=128,kernel_size=(3,3),activation="relu",padding="same"))
# model.add(Conv2D(filters=128,kernel_size=(3,3),activation="relu",padding="same"))
# model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

# model.add(Conv2D(filters=256,kernel_size=(3,3),activation="relu",padding="same"))
# model.add(Conv2D(filters=256,kernel_size=(3,3),activation="relu",padding="same"))
# model.add(Conv2D(filters=256,kernel_size=(3,3),activation="relu",padding="same"))
# model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

# model.add(Conv2D(filters=512,kernel_size=(3,3),activation="relu",padding="same"))
# model.add(Conv2D(filters=512,kernel_size=(3,3),activation="relu",padding="same"))
# model.add(Conv2D(filters=512,kernel_size=(3,3),activation="relu",padding="same"))
# model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))


# model.add(Conv2D(filters=512,kernel_size=(3,3),activation="relu",padding="same"))
# model.add(Conv2D(filters=512,kernel_size=(3,3),activation="relu",padding="same"))
# model.add(Conv2D(filters=512,kernel_size=(3,3),activation="relu",padding="same"))
# model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

# model.add(Flatten())
# model.add(BatchNormalization())
# model.add(Dense(units=4096,activation="relu"))
# model.add(Dense(units=4096,activation="relu"))
# model.add(Dense(units=100,activation="softmax"))


# model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=["accuracy"])
# model.summary()



# #early stopping
# from keras.callbacks import EarlyStopping
# early=EarlyStopping(monitor='val_accuracy',min_delta=0,patience=30,verbose=1,mode='auto')

# history=model.fit(x_train,y_train,batch_size=100,validation_data=(x_val,y_val),epochs=15,callbacks=[early])


# score=model.predict(x_test).argmax(-1)

# from sklearn.metrics import confusion_matrix
# from sklearn.metrics import precision_score,accuracy_score,recall_score
# matrix1=confusion_matrix(y_test.argmax(axis=1),score)
# print(matrix1)
# p_score=precision_score(y_test.argmax(axis=1),score,average='weighted')
# acc_score=accuracy_score(y_test.argmax(axis=1),score)
# r_score=recall_score(y_test.argmax(axis=1),score,average='weighted')

# print('precision',p_score)
# print('accuracy',acc_score)
# print('recall',r_score)